# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 13ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [10]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [11]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [12]:
# Train the model

normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model


Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [13]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [14]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 12ms/step - loss: 0.4909 - mse: 0.4909 - val_loss: 0.1612 - val_mse: 0.1612
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2271 - mse: 0.2271 - val_loss: 0.1285 - val_mse: 0.1285
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1559 - mse: 0.1559 - val_loss: 0.1175 - val_mse: 0.1175
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1365 - mse: 0.1365 - val_loss: 0.1329 - val_mse: 0.1329
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1275 - mse: 0.1275 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1130 - mse: 0.1130 - val_loss: 0.1246 - val_mse: 0.1246
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1037 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1255 - val_mse: 0.1255
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0225 - mse: 0.0225 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1257 - val_mse: 0.1257
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1246 - val_mse: 0.1246
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0205 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1244 - val_mse: 0.1244
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1233 - val_mse: 0.1233
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0082 - mse: 0.0082


[0.008237401023507118, 0.008237401023507118]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [16]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1241 - mse: 0.1241


[0.12410231679677963, 0.12410231679677963]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [17]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data

np.sqrt(mean_squared_error(y_val, y_val_pred))

27683.527809719337

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [18]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])


# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.5030 - mse: 0.5030 - val_loss: 0.2333 - val_mse: 0.2333
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2649 - mse: 0.2649 - val_loss: 0.1918 - val_mse: 0.1918
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2164 - mse: 0.2164 - val_loss: 0.1588 - val_mse: 0.1588
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1910 - mse: 0.1910 - val_loss: 0.1475 - val_mse: 0.1475
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1685 - mse: 0.1685 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1517 - mse: 0.1517 - val_loss: 0.1398 - val_mse: 0.1398
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1373 - mse: 0.1373 - val_loss: 0.1405 - val_mse: 0.1405
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1309 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0277 - mse: 0.0277 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 62/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 63/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0265 - mse: 0.0265 - val_loss: 0.1419 - val_mse: 0.1419
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.1430 - val_mse: 0.1430
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0245 - mse: 0

Epoch 123/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 124/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1429 - val_mse: 0.1429
Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1403 - val_mse: 0.1403
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0092 - mse: 0.0092


[0.009188980795443058, 0.009188980795443058]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [20]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1435 - mse: 0.1435


[0.14348411560058594, 0.14348411560058594]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [21]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 12ms/step - loss: 0.4392 - mse: 0.4392 - val_loss: 0.1791 - val_mse: 0.1791
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2232 - mse: 0.2232 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1824 - mse: 0.1824 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1542 - mse: 0.1542 - val_loss: 0.1144 - val_mse: 0.1144
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1394 - mse: 0.1394 - val_loss: 0.1244 - val_mse: 0.1244
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1187 - mse: 0.1187 - val_loss: 0.1097 - val_mse: 0.1097
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1117 - 

33/33 [==============================] - 0s 5ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 62/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1156 - val_mse: 0.1156
Epoch 63/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1159 - val_mse: 0.1159
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1163 - val_mse: 0.1163
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1174 - val_mse: 0.1174
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0194 - mse: 0.0194 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0200 - mse: 0

Epoch 123/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 124/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1184 - val_mse: 0.1184
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1198 - val_mse: 0.1198
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0071 - mse: 0.0071


[0.00707729859277606, 0.00707729859277606]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1204 - mse: 0.1204


[0.12042596936225891, 0.12042596936225891]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [24]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])


# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.3424 - mse: 0.3424 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1802 - mse: 0.1802 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1399 - mse: 0.1399 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1124 - mse: 0.1124 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0884 - mse: 0.0884 - val_loss: 0.0945 - val_mse: 0.0945
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0827 - mse: 0.0827 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0770 - mse: 0.0770 - val_loss: 0.0970 - val_mse: 0.0970
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0629 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0074 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1034 - val_mse: 0.1034
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0891 - val_mse: 0.0891
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0960 - val_mse: 0.0960
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1023 - val_mse: 0.1023
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0962 - val_mse: 0.0962
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0879 - val_mse: 0.0879
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0938 - val_mse: 0.0938
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0028 - mse: 0.0028


[0.0028449485544115305, 0.0028449485544115305]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [26]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0952 - mse: 0.0952


[0.09517792612314224, 0.09517792612314224]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [27]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.4209 - mse: 0.4209 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1569 - mse: 0.1569 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1045 - mse: 0.1045 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0773 - mse: 0.0773 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0577 - mse: 0.0577 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0435 - mse: 0.0435 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0338 - mse: 0.0338 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0309 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1002 - val_mse: 0.1002
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.0954 - val_mse: 0.0954
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0033 - mse: 0

33/33 [==============================] - 0s 4ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.0887 - val_mse: 0.0887
Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0922 - val_mse: 0.0922
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 9.9747e-04 - mse: 9.9747e-04 - val_loss: 0.0886 - val_mse: 0.0886
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0930 - val_mse: 0.0930
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0890 - val_mse: 0.0890
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0949 - val_mse: 0.0949
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0828 - val_mse: 0.0828
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0032 - mse: 0.0032


[0.003246620064601302, 0.003246620064601302]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [29]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0923 - mse: 0.0923


[0.09227573126554489, 0.09227573126554489]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 3ms/step - loss: 0.1419 - mse: 0.1419


[0.14190900325775146, 0.14190900325775146]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [31]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29603.050960124823

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.